## Environment Setup - LangSmith

In [4]:
!pip install langsmith openai ollama

In [10]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true' # enables encoding
os.environ['LANGCHAIN_API_KEY'] = "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"

In [12]:
import os
os.environ['LANGCHAIN_PROJECT'] = 'Test'

## Manually Curated Dataset

In [22]:
import pandas as pd

# QA
inputs = [
    "How many tokens was DBRX pre-trained on?",
    "Is DBRX a MOE model and how many parameters does it have?",
    "How many GPUs was DBRX trained on and what was the connectivity between GPUs?",
]

outputs = [
    "DBRX was pre-trained on 12 trillion tokens of text and code data.",
    "Yes, DBRX is a fine-grained mixture-of-experts (MoE) architecture with 132B total parameters.",
    "DBRX was trained on 3072 NVIDIA H100s connected by 3.2Tbps Infiniband",
]

# Dataset
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]
df = pd.DataFrame(qa_pairs)

# Write to csv
csv_path = "F:\\LangSmith\\DBRX_eval.csv"
df.to_csv(csv_path, index=False) # Now Upload this Dataset from device to LangSmith

In [17]:
from langsmith import Client

client = Client()
dataset_name = "DBRX"

# Store
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="QA pairs about DBRX model.",
)
client.create_examples(
    inputs=[{"question": q} for q in inputs],
    outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id,
)

{'example_ids': ['c8b8d557-a1c7-48a2-9a5c-8083b6efd974',
  'df818c21-02de-478c-93dc-013a21dbe990',
  '2e1ec321-d8b9-4fd9-87c9-0a87841696ec'],
 'count': 3}

In [19]:
new_questions = [
    "What is the context window of DBRX Instruct?",
]

new_answers = [
    "DBRX Instruct was trained with up to a 32K token context window.",
]

# See updated version in the UI
client.create_examples(
    inputs=[{"question": q} for q in new_questions],
    outputs=[{"answer": a} for a in new_answers],
    dataset_id=dataset.id,
)

{'example_ids': ['3d6cd2f6-edc5-4c90-a123-5186cfc39909'], 'count': 1}